# Grid of Resistors (DistributedArrays version)
## Stencil computation with DArrays

Again we provide the reference code but 

In [ ]:
using BenchmarkTools
using JuliaRunClient
initializeCluster(2)
# addprocs(2) # local version
using DistributedArrays

We'll resume where the sequential version of the grid of resistors exercise ended. It is critical for good parallel performance to identify data dependencies such that communication can be minimized. Hence we will use the result of the exercise where you rewrote the sequential version such that the loop body for the loop over repetitions consisted of two function calls such that everything within each of the functions could be executed concurrently. A version of the solution is shown below.

In [ ]:
function stencil(v, b, ω, i, j)
    return (1 - ω)*v[i, j] +
        ω*(v[i + 1, j] + v[i - 1, j] + v[i, j + 1] + v[i, j - 1] + 
            b[i, j])/4
end

function apply_stencil_eeoo!(v, b, ω, n)
    # even-even and odd-odd
    for j in 1:n
        for i in 1:(n - 1)
            v[2*i, 2*j]         = stencil(v, b, ω, 2*i    , 2*j)
            v[2*i + 1, 2*j + 1] = stencil(v, b, ω, 2*i + 1, 2*j + 1)
        end
        v[2*n, 2*j] = stencil(v, b, ω, 2*n, 2*j)
    end
    return v
end
                
function apply_stencil_eooe!(v, b, ω, n)
    # even-odd and odd-even
    for j in 1:n
        for i in 1:(n - 1)
            v[2*i, 2*j + 1] = stencil(v, b, ω, 2*i    , 2*j + 1)
            v[2*i + 1, 2*j] = stencil(v, b, ω, 2*i + 1, 2*j)
        end
        v[2*n, 2*j + 1] = stencil(v, b, ω, 2*n, 2*j + 1)
    end
    return v
end

function compute_resistance(n, reps = 100)
    # assume n and omega already defined or take
    # the following values for the optimal omega
    μ = (cos(π/(2*n)) + cos(π/(2*n + 1)))/2
    ω = 2*(1 - sqrt(1 - μ^2))/μ^2
    # (See page 409 of Strang Intro to Applied Math , this is equation 16)

    # Initialize voltages
    v = zeros(2*n + 1, 2*n + 2)

    # Define Input Currents
    b = copy(v)
    b[n + 1, (n + 1):(n + 2)]  = [1 -1]

    # Jacobi Steps
    for k in 1:reps

        apply_stencil_eeoo!(v, b, ω, n)
        apply_stencil_eooe!(v, b, ω, n)
        
    end
# Compute resistance = v_A - v_b = 2 v_A
    r = 2*v[n + 1, n + 1]
    return v, r, b
end

In [ ]:
@btime compute_resistance(400)[2]

This problam (as is the case for most parallel problems) can be decomposed into an embarrassingly parallel part and a part with communication. Our strategy for utilizing multiple processes to speed up the computation is to split the `v` array into equal chunks over the number of processes. This can be done with `DArray` from the `DistributedArrays` package.

The problem with splitting up `v` is that some of the stencil computations now have a dependency across processes. The solutions is to introduce  ghost regions around each array that hold values that identical to values on the neighborring arrays.

This is easiest to demonstrate for a small array. We can use the `compute_resistance` function to compute a `v` matrix of size $5\times6$.

In [ ]:
v, r, b = compute_resistance(2)
v

We can now use the `remotecall` function to create chunks on the processes 2 and 3.

In [ ]:
r1 = remotecall(2) do
    v[:, 1:4]
end
r2 = remotecall(3) do
    v[:, 3:6]
end

There is a `DArray` contructor that takes an array of remote references as input and constructs a `DArray`.

In [ ]:
dv = DArray([r1 r2])

Adjust the two `apply_stencil_xxxx!` functions to use the size of `v` instead of `n` to determine the loop ranges

In [ ]:
@everywhere function stencil(v, b, ω, i, j)
    return  (1 - ω)*v[i, j] +
        ω*(v[i + 1, j] + v[i - 1, j] + v[i, j + 1] + v[i, j - 1] + 
            b[i, j])/4
end

@everywhere function apply_stencil_eeoo!(v, b, ω)
    # even-even and odd-odd
    m = div(size(v, 1), 2)     # hard coded distribution
    n = div(size(v, 2), 2) - 1 # hard coded distribution
    for j in 1:n
        for i in 1:(m - 1)
            v[2*i, 2*j]         = stencil(v, b, ω, 2*i    , 2*j)
            v[2*i + 1, 2*j + 1] = stencil(v, b, ω, 2*i + 1, 2*j + 1)
        end
        v[2*m, 2*j] = stencil(v, b, ω, 2*m, 2*j)
    end
    return nothing
end
                
@everywhere function apply_stencil_eooe!(v, b, ω)
    # even-odd and odd-even
    m = div(size(v, 1), 2)     # hard coded distribution
    n = div(size(v, 2), 2) - 1 # hard coded distribution
    for j in 1:n
        for i in 1:(m - 1)
            v[2*i, 2*j + 1] = stencil(v, b, ω, 2*i    , 2*j + 1)
            v[2*i + 1, 2*j] = stencil(v, b, ω, 2*i + 1, 2*j)
        end
        v[2*m, 2*j + 1] = stencil(v, b, ω, 2*m, 2*j + 1)
    end
    return nothing
end

Before turning to the distributed version, we will first introduce the `localpart` function which will be needed for the parallel version. The function simply returns the local memory part of a distributed array.

**Exercise** Allocate a distributed random matrix with `drand(6,8)` and print the `localpart`s.

Below is a distributed version of the resistance calculation. (Be aware that it is just a toy model to show how `remotecall`s work in Julia.) The code has the same structure as the sequential version so they are easier to compare.

In [ ]:
function compute_resistance_dist(n, reps = 100)
    # To make things simple, we only allow even n
    iseven(n) || throw(ArgumentError("only even n allowed"))
    
    # assume n and omega already defined or take
    # the following values for the optimal omega
    μ = (cos(π/(2*n)) + cos(π/(2*n + 1)))/2
    ω = 2*(1 - sqrt(1 - μ^2))/μ^2
    # (See page 409 of Strang Intro to Applied Math , this is equation 16)

    dv = dzeros((2*n + 1, 2*n + 2 + 2), [2, 3])
    db = copy(dv)
    
    remotecall_fetch(2) do
        DistributedArrays.localpart(db)[n + 1, n + 1] =  1
        DistributedArrays.localpart(db)[n + 1, n + 2] = -1
    end
    remotecall_fetch(3) do
        DistributedArrays.localpart(db)[n + 1, 1] =  1
        DistributedArrays.localpart(db)[n + 1, 2] = -1
    end
    
    # Jacobi Steps
    for k in 1:reps

        # even-even and odd-odd
        remotecall_fetch(2) do
            apply_stencil_eeoo!(DistributedArrays.localpart(dv), DistributedArrays.localpart(db), ω)
        end
        remotecall_fetch(3) do
            apply_stencil_eeoo!(DistributedArrays.localpart(dv), DistributedArrays.localpart(db), ω)
        end

        # Make ghost regions consistent
        tmp = remotecall_fetch(2) do
            DistributedArrays.localpart(dv)[3:2:(2*n - 1), end - 1]
        end
        tmp = remotecall_fetch(3) do
            DistributedArrays.localpart(dv)[3:2:(2*n - 1), 1] = tmp
            DistributedArrays.localpart(dv)[2:2:(2*n), 2]
        end
        remotecall_fetch(2) do
            DistributedArrays.localpart(dv)[2:2:(2*n), end] = tmp
            nothing
        end

        # even-odd and odd-even
        remotecall_fetch(2) do
            apply_stencil_eooe!(DistributedArrays.localpart(dv), DistributedArrays.localpart(db), ω)
        end
        remotecall_fetch(3) do
            apply_stencil_eooe!(DistributedArrays.localpart(dv), DistributedArrays.localpart(db), ω)
        end

        # Make ghost regions consistent
        tmp = remotecall_fetch(2) do
            DistributedArrays.localpart(dv)[2:(2*n), end - 1]
        end
        tmp = remotecall_fetch(3) do
            DistributedArrays.localpart(dv)[2:(2*n), 1] = tmp
            DistributedArrays.localpart(dv)[2:(2*n + 1), 2]
        end
        remotecall_fetch(2) do
            DistributedArrays.localpart(dv)[2:(2*n + 1), end] = tmp
            nothing
        end
    end
# Compute resistance = v_A - v_b = 2 v_A
    r = 2*dv[n + 1, n + 1]
    return dv, r, db
end

**Exercise** Read through the code and make sure you understand. Can you tell why some of the anonymous functions return `nothing`?

**Exercise** Time the distributed version and compare to the sequential version. This version isn't fast at all. What is wrong?

**Exercise** Add `@sync` and `@async` statements to the code to speed it up. Increase the size of the problem. Can you make the distributed version faster than the sequential?

In [ ]:
@time dv, dr, db = compute_resistance_dist(n)

In [ ]:
# releaseCluster();